In [ ]:
%pip install termcolor

In [9]:
from sempy import fabric
from termcolor import cprint
from datetime import datetime
import time
import math

StatementMeta(, 887eab43-33a1-4658-89f8-56c313514548, 21, Finished, Available, Finished)

## Enter required details in the below cell

In [ ]:
datasetId = "" #DatasetID
workspaceId = "" #WorkspaceID
tableName = "" #Table Name
batchSize = 4 #number of partitions to refresh in a single batch
delay = 180 #delay between each batch refresh in seconds
payload = {
    "type": "Full",
    "commitMode": "transactional",
    "maxParallelism": 2,
    "retryCount": 1,
    "timeout": "02:00:00",
    "objects": [],
    "applyRefreshPolicy": False
}
fullEndPoint = f"https://api.powerbi.com/v1.0/myorg/groups/{workspaceId}/datasets/{datasetId}/refreshes"    

StatementMeta(, 887eab43-33a1-4658-89f8-56c313514548, 22, Finished, Available, Finished)

In [11]:
df_partitions = fabric.list_partitions(dataset=datasetId, workspace=workspaceId ,table=tableName)
paritions = df_partitions["Partition Name"].tolist()

StatementMeta(, 887eab43-33a1-4658-89f8-56c313514548, 23, Finished, Available, Finished)

In [12]:
def refresh_partitions_in_batches(datasetId:str, workspaceId:str,tableName:str ,partitions:list):
    batchCount = math.ceil(len(partitions) / batchSize) if len(partitions) > 0 else 0
    client = fabric.FabricRestClient()
    batch_num = 0

    def getrefreshStatus():
        response = client.get(f"{fullEndPoint}?$top=1")
        return response.json().get("value",{})[0].get("status","")

    def postRefresh(payload):
        response = client.post(fullEndPoint,json=payload)
        return response.status_code
    
    # wait until service is idle before triggering this batch
    while True and batch_num == 0:
        status = getrefreshStatus()
        cur_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        cprint(f"Before triggering the first batch at {cur_time} last refresh status: {status}",'blue')
        if status in ["Completed", "Failed", "None"]:
            cprint("🏁 All to set to start",'green')
            break
        cprint("⏱️ Waiting for {delay} seconds before checking status...",'yellow')
        time.sleep(delay)

    cprint(f"📋 Total Batches to process: {batchCount}",'blue')

    for batch_num in range(batchCount):
        objects = []
        temp = partitions[batch_num * batchSize : (batch_num + 1) * batchSize]
        for i in temp:
            objects.append({"table":tableName, "partition": i})
        payload["objects"] = objects
 
        try:
            # trigger the refresh for this batch
            cprint(f"🏹 Triggering for batch {batch_num + 1} with partitions: {temp}",'yellow')
            resp = postRefresh(payload=payload)
            if resp != 202:
                cprint(f"🔴 Failed to trigger batch {batch_num + 1 }, response: {resp}",'red')
                # decide whether to retry or continue to next batch
                continue
            else:
                cprint(f"🎯 Batch {batch_num + 1} triggered successfully.",'blue')

            # poll until this triggered refresh finishes
            while True:
                cprint(f"⏱️ Waiting for {delay} seconds before checking status...",'yellow')
                time.sleep(delay)
                status = getrefreshStatus()
                cur_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                cprint(f"🟢 Current status {cur_time}: {status}",'yellow')
                if status in ["Completed", "Failed", "None"]:
                    cprint(f"🏎️ Batch {batch_num + 1} finished with status: {status}",'green' if status=="Completed" else 'red')
                    if batch_num + 1 == batchCount:
                        cprint(f"✅ All batches processed.",'blue')
                    break

        except Exception as e:
            cprint(f"🔴 Error while running the API: {e}",'red')

StatementMeta(, 887eab43-33a1-4658-89f8-56c313514548, 24, Finished, Available, Finished)

In [ ]:
refresh_partitions_in_batches(datasetId=datasetId,workspaceId=workspaceId,tableName=tableName,partitions=paritions)